In [1]:
import selenium
from selenium import webdriver
import os
import time
import requests
import hashlib
import io
# This is the path I use

# DRIVER_PATH = '/home/vernon/Google_Chrome_Driver/chromedriver'
# Put the path for your ChromeDriver here

DRIVER_PATH = './chromedriver_windows32/chromedriver.exe'

# wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [2]:
#wd.get('https://google.com.au')

In [3]:
#search_box = wd.find_element_by_css_selector('input.gLFyf')
#search_box.send_keys('Dogs')

In [4]:
x = set([1, 2, 3])
x.add(5)
for i in x:
    print(i)

1
2
3
5


In [5]:
###########################################################################################
###########################################################################################
###########################################################################################

In [6]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    retries = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        if retries > 200:
            break
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(5)
            retries += 1
            if retries > 20:
                break
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)
    return image_urls

In [7]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        #
        #
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=100)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [8]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5,split='train'):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')),split)

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [9]:
import selenium
from selenium import webdriver
import requests
import os
import io
import time
import hashlib
from PIL import Image

search_term = 'cat with dog photo'

search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH,
    number_images=200
)

Found: 100 search results. Extracting links from 0:100
Found: 146 image links, looking for more ...
Found: 212 search results. Extracting links from 100:212
Found: 294 image links, looking for more ...
Found: 312 search results. Extracting links from 212:312
Found: 428 image links, looking for more ...
Found: 412 search results. Extracting links from 312:412
Found: 569 image links, looking for more ...
Found: 512 search results. Extracting links from 412:512
Found: 569 image links, looking for more ...
Found: 100 search results. Extracting links from 512:100
Found: 569 image links, looking for more ...
Found: 200 search results. Extracting links from 100:200
Found: 710 image links, looking for more ...
Found: 312 search results. Extracting links from 200:312
Found: 851 image links, looking for more ...
Found: 355 search results. Extracting links from 312:355
Found: 895 image links, looking for more ...
Found: 355 search results. Extracting links from 355:355
Found: 895 image links, loo

KeyboardInterrupt: 